# Prepare model for CoCo


In [6]:
# Setup 
import os
CWD = os.getcwd() + "/"
originalImagesPath     = CWD + '../../data/coco/originalImages'
preprocessedImagesPath = CWD + '../../data/coco/processedImages/'

caffe_root = '/Users/Grendel/caffe/'

vgg_ilsvrc_19_layoutFileName = caffe_root + 'models/vgg_ilsvrc_19/VGG_ILSVRC_19_layers_deploy.prototxt'
vgg_ilsvrc_19_modelFileName  = caffe_root + 'models/vgg_ilsvrc_19/VGG_ILSVRC_19_layers.caffemodel'

dataPath        = CWD + '../../data/coco/'
annotation_path = dataPath + 'annotations/'
annotation_tr   = annotation_path + 'captions_train2014.json'
splitFileName   = dataPath + 'dataset_coco.json'

tr_data_path    = dataPath + 'train2014/'
te_data_path    = dataPath + 'test2014/'
val_data_path   = dataPath + 'val2014/'

experimentPrefix = '.exp1'



In [7]:
# Import

import pdb
from sys import stdout
import scipy
import  cPickle as pickle

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import pandas as pd
import nltk

In [8]:
caffe.set_device(0)
caffe.set_mode_gpu()

net = caffe.Net(vgg_ilsvrc_19_layoutFileName,
                vgg_ilsvrc_19_modelFileName,
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

In [9]:
# Create file list 
# coco.devImages.txt 
# coco.trainImages.txt 
# coco.testImages.txt

# import json
# from pprint import pprint


# with open(splitFileName) as f:
#     data = json.load(f)

# df = pd.DataFrame(data['images'])

# files = [ 'dev','test','train']

# dataDict = {}

# dataDict['dev'] = df[df.split == 'val']
# dataDict['test'] = df[df.split == 'test']
# dataDict['restval'] = df[df.split == 'restval']
# dataDict['train'] = df[df.split == 'train']

# for f in files:
#     dataDict[f]['filename'].to_csv(dataPath + 'coco.' + f + 'Images.txt',index=False)
    

# def buildCapDict(sentences):
#     return [s[u'raw'] for s in sentences ]

# df['captions'] = df.apply(lambda row: buildCapDict(row['sentences']), axis=1)

# capDict = df.loc[:,['filename', 'captions']].set_index('filename').to_dict()

# capDict = capDict['captions']

"""
Use:
    ls <your directory> > <the file you want to save to>
"""

'\nUse:\n    ls <your directory> > <the file you want to save to>\n'

In [10]:
# Build dictionary
# corpus = df['captions'].values
# corpus2 = [' '.join(c) for c in corpus]
# corpus3 = ' '.join(corpus2)

# Make the dictionary form the captisonw
import json
with open(annotation_tr,'r') as f:
    caps_notes = json.load(f)
    corpus3 = ' '.join([note['caption'] for note in caps_notes['annotations']])
words = nltk.FreqDist(corpus3.split()).most_common()

wordsDict = {words[i][0]:i+2 for i in range(len(words))}

with open(dataPath + 'dictionary.pkl', 'wb') as f:
    pickle.dump(wordsDict, f)

In [11]:
words[:10]

[(u'a', 443447),
 (u'A', 241099),
 (u'on', 148118),
 (u'of', 141775),
 (u'in', 127752),
 (u'the', 118869),
 (u'with', 106930),
 (u'and', 98049),
 (u'is', 67163),
 (u'man', 48108)]

In [ ]:
# set net to batch size of 50
# net.blobs['data'].reshape(10,3,224,224)

files = [ 'val','test','train']

for fname in files:
    print fname 
    f = open(dataPath + "annotations/" + 'coco.' + fname + 'Images.txt')
    counter = 0
    
    imageList = [i for i in f]
    numImage = len(imageList)
    
    result = np.empty((numImage, 100352))

    for i in range(numImage):
        fn = imageList[i].rstrip()
        net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image( dataPath + fname + "2014/" +  fn))
        out = net.forward()
        feat = net.blobs['conv5_4'].data[0]
        reshapeFeat = np.swapaxes(feat, 0,2)
        reshapeFeat2 = np.reshape(reshapeFeat,(1,-1))
        
        counter += 1
        stdout.write("\r%d" % counter)
        stdout.flush()
        result[i,:] = reshapeFeat2
        
    print result.shape
    
    resultSave = scipy.sparse.csr_matrix(result)
    resultSave32 = resultSave.astype('float32')
    
    if fname == 'train':
        np.savez(dataPath + 'coco_feature.' + fname + experimentPrefix, data=resultSave32.data, indices=resultSave32.indices, indptr=resultSave32.indptr, shape=resultSave.shape)
    else:
        fileName = open(dataPath + 'coco_feature.' + fname + experimentPrefix + '.pkl','wb')
        pickle.dump(resultSave32, fileName ,-1)
        fileName.close()



In [17]:
np.savez(dataPath + 'coco_feature.' + fname + experimentPrefix, data=resultSave32.data, indices=resultSave32.indices, indptr=resultSave32.indptr, shape=resultSave.shape)
  


In [18]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [19]:
files = ['test', 'dev']

for name in files:
    counter = 0
    feat = pickle.load(open(dataPath + 'coco_feature.' + name + experimentPrefix + '.pkl','rb'))
    filenames = open(dataPath + 'coco.' + name + 'Images.txt')
    cap = []
    for imageFile in filenames:
        imageFile = imageFile.rstrip()
        for sen in capDict[imageFile]:
            cap.append([sen.rstrip(), counter])
        counter += 1
    saveFile = open(dataPath + 'coco_align.' + name + experimentPrefix + '.pkl', 'wb')
    pickle.dump(cap, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(feat, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    saveFile.close()
            
    
    

In [20]:
files = ['train']

for name in files:
    counter = 0
#     feat = pickle.load(open(dataPath + 'coco_feature.' + name +'.pkl','rb'))
    filenames = open(dataPath + 'coco.' + name + 'Images.txt')
    cap = []
    for imageFile in filenames:
        imageFile = imageFile.rstrip()
        for sen in capDict[imageFile]:
            cap.append([sen.rstrip(), counter])
        counter += 1
    saveFile = open(dataPath + 'coco_align.' + name + experimentPrefix + '.pkl', 'wb')
    pickle.dump(cap, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
#     pickle.dump(feat, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    saveFile.close()

In [21]:
wordsDict['Two']

22

In [22]:
resultSave32

<82783x100352 sparse matrix of type '<type 'numpy.float32'>'
	with 502170223 stored elements in Compressed Sparse Row format>